In [1]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn.model_selection import train_test_split


In [3]:
import csv

header = [f'column_{str(i)}' for i in range(12)]
header.append('label')

with open('Train.txt', 'r') as txtfile, open('Train.csv', 'w', newline='') as csvfile:
    txt_reader = csv.reader(txtfile, delimiter=',')
    
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(header)
    
    for row in txt_reader:
        csv_writer.writerow(row)

FileNotFoundError: [Errno 2] No such file or directory: 'train.txt'

In [2]:
h2o.init(port=8001)
# h2o.cluster().shutdown(prompt=True) 

Checking whether there is an H2O instance running at http://localhost:8001..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.381-b09, mixed mode)
  Starting server from C:\Users\alcarachin\Desktop\autoML_test\venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ALCARA~1\AppData\Local\Temp\tmpz819ykor
  JVM stdout: C:\Users\ALCARA~1\AppData\Local\Temp\tmpz819ykor\h2o_alcarachin_started_from_python.out
  JVM stderr: C:\Users\ALCARA~1\AppData\Local\Temp\tmpz819ykor\h2o_alcarachin_started_from_python.err


C:\Users\alcarachin\Desktop\autoML_test\venv\lib\site-packages\h2o\backend\server.py:385: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Server is running at http://127.0.0.1:8001
Connecting to H2O server at http://127.0.0.1:8001 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 13 days
H2O_cluster_name:,H2O_from_python_alcarachin_4mvakw
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,247.5 Mb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [3]:
data = pd.read_csv('train.csv')
label = 'track_genre'
data = data.drop(columns=['Unnamed: 0', 'track_id'])
data.head()

train_data, test_data = train_test_split(data, test_size=0.8, shuffle=True)
train_data.head()

train_data = h2o.H2OFrame(train_data)
test_data = h2o.H2OFrame(test_data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
x = train_data.columns
y = "track_genre"
x.remove(y)

train_data[y] = train_data[y].asfactor()
test_data[y] = test_data[y].asfactor()

aml = H2OAutoML(max_models=3, seed=1)
aml.train(x=x, y=y, training_frame=train_data)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)

aml.leader

In [ ]:
# SAVE MODEL

model_path = h2o.save_model(model=aml, path="./tmp/mymodel", force=True)

In [ ]:
# LOAD MODEL

model_path = h2o.save_model(model=aml, path="./tmp/mymodel", force=True)
saved_model = h2o.load_model(model_path)

In [ ]:
preds = aml.leader.predict(test_data)

In [ ]:
acc = sum((preds['predict'] == test_data[y]).as_data_frame()['predict'].to_numpy()) / len(test_data)
print(f'Accuracy on test data: {acc}')